In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
print("Pandas version: ", pd.__version__)

Pandas version:  2.0.3


In [2]:
current_path = 'jupyter/ya'

# current_path = 'iter-20230609/by_balabanov'
df_final_filename = f'df_final.csv'

dfFinal = pd.read_csv(df_final_filename)

print('df_final: ', dfFinal.shape)

df_final:  (4693, 40)


In [3]:
dfFinal.dropna(axis=1,inplace=True)

In [4]:
y = dfFinal['type']
drop_col = ['type','proto','diag_step_fiat','diag_step_biat','diag_steps']
X  = dfFinal.drop(columns=drop_col)

#### Разобьем данные на подопытные и проверочные

In [5]:
X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        test_size=0.3,
                                                        random_state=42,
                                                        stratify=None)

## CNN

In [6]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    roc_auc_score,
    confusion_matrix,
    accuracy_score,
    classification_report,
)

In [8]:
!pip3 install tensorflow

     |████████████████████████████████| 479.6 MB 37 kB/s  eta 0:00:011     |██████████▎                     | 154.2 MB 21.6 MB/s eta 0:00:16
     |████████████████████████████████| 65 kB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.4 MB 19.4 MB/s eta 0:00:01
     |████████████████████████████████| 81 kB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.8 MB 24.9 MB/s eta 0:00:01
     |████████████████████████████████| 440 kB 15.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 13.8 MB/s eta 0:00:01
     |████████████████████████████████| 130 kB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 14.7 MB/s eta 0:00:01
     |████████████████████████████████| 311 kB 26.1 MB/s eta 0:00:01
     |████████████████████████████████| 17.3 MB 7.0 MB/s eta 0:00:011
     |████████████████████████████████| 22.9 MB 14.1 MB/s eta 0:00:01    |███████████                     | 7.9 MB 22.3 MB/s eta 0:00:01
     |████████████████████████████

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, Conv1D, MaxPooling1D, AveragePooling1D
from tensorflow.keras.metrics import AUC, CategoricalAccuracy, CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

ModuleNotFoundError: No module named 'tensorflow'

## 1. CNN с двумя признаками на пакет


In [27]:
metrics = [AUC(), CategoricalAccuracy(), CategoricalCrossentropy()]

# количества нейронов в слоях
INPUT_SIZE = 1  # TODO Должно быть 2 # количество признаков для пакета - iat и size
STEPS = X.shape[1]  # кол-во пакетов

In [28]:
STEPS

20

In [29]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=11, activation="relu",
          padding="same", input_shape=(STEPS, INPUT_SIZE)))
model.add(AveragePooling1D())
model.add(Conv1D(filters=64, kernel_size=5, activation="relu", padding="same"))
model.add(AveragePooling1D())
model.add(Conv1D(filters=128, kernel_size=3,
          activation="relu", padding="same"))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [30]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=metrics)

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 20, 32)            384       
                                                                 
 average_pooling1d_2 (Avera  (None, 10, 32)            0         
 gePooling1D)                                                    
                                                                 
 conv1d_4 (Conv1D)           (None, 10, 64)            10304     
                                                                 
 average_pooling1d_3 (Avera  (None, 5, 64)             0         
 gePooling1D)                                                    
                                                                 
 conv1d_5 (Conv1D)           (None, 5, 128)            24704     
                                                                 
 flatten_1 (Flatten)         (None, 640)              

In [32]:
model.fit(X_train, y_train, batch_size=128, epochs=20, verbose=1)

Epoch 1/20


/opt/bitnami/miniconda/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


26/26 [==============================] - 2s 16ms/step - loss: -132.2391 - auc_1: 0.5198 - categorical_accuracy: 1.0000 - categorical_crossentropy: 0.0000e+00
Epoch 2/20
26/26 [==============================] - 0s 16ms/step - loss: -7123.5732 - auc_1: 0.5000 - categorical_accuracy: 1.0000 - categorical_crossentropy: 0.0000e+00
Epoch 3/20
26/26 [==============================] - 0s 18ms/step - loss: -171528.8125 - auc_1: 0.5000 - categorical_accuracy: 1.0000 - categorical_crossentropy: 0.0000e+00
Epoch 4/20
26/26 [==============================] - 0s 17ms/step - loss: -2046621.6250 - auc_1: 0.5000 - categorical_accuracy: 1.0000 - categorical_crossentropy: 0.0000e+00
Epoch 5/20
26/26 [==============================] - 0s 15ms/step - loss: -14228357.0000 - auc_1: 0.5000 - categorical_accuracy: 1.0000 - categorical_crossentropy: 0.0000e+00
Epoch 6/20
26/26 [==============================] - 0s 15ms/step - loss: -65152284.0000 - auc_1: 0.5000 - categorical_accuracy: 1.0000 - categorical_cros

In [33]:
y_test

1550    1
3427    2
4327    2
584     1
3715    2
       ..
3936    2
4570    2
925     1
4634    2
3527    2
Name: type, Length: 1408, dtype: int64

In [34]:
y_predicted_train = model.predict(X_train)

103/103 [==============================] - 1s 7ms/step


In [35]:
y_predicted_test = model.predict(X_test)

44/44 [==============================] - 0s 6ms/step


In [36]:
y_predicted_test

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)

In [37]:
print("\n" + "#" * 80)
print(
    "Train accuracy:", accuracy_score(y_train, np.where(y_predicted_train > 0.5, 1, 0))
)
print("Test accuracy:", accuracy_score(y_test, np.where(y_predicted_test > 0.5, 1, 0)))
print("Confusion Matrix:")
print(confusion_matrix(y_test, np.where(y_predicted_test > 0.5, 1, 0)))




################################################################################
Train accuracy: 0.3278538812785388
Test accuracy: 0.3536931818181818
Confusion Matrix:
[[  0 452   0]
 [  0 498   0]
 [  0 458   0]]
